## Examples

In [1]:
from thecodecrate_pipeline import (
    Pipeline,
)


pipeline = (
    (Pipeline[int]())
    .pipe(lambda payload: payload + 1)
    .pipe(lambda payload: payload + 1)
)

# outputs 3
await pipeline.process(1)

3

In [2]:
from thecodecrate_pipeline import (
    StageInterface,
)


class TimesTwoStage(StageInterface[int]):
    async def __call__(self, payload: int) -> int:
        return payload * 2


class AddOneStage(StageInterface[int]):
    async def __call__(self, payload: int) -> int:
        return payload + 1


my_pipeline = (Pipeline[int]()).pipe(TimesTwoStage()).pipe(AddOneStage())

# returns 21
await my_pipeline.process(10)

21

In [3]:
class TimesThreeStage(StageInterface[int]):
    async def __call__(self, payload: int) -> int:
        return payload * 3


class AddFiveStage(StageInterface[int]):
    async def __call__(self, payload: int) -> int:
        return payload + 5


pipeline = (
    (Pipeline[int]())
    .pipe(TimesThreeStage())
    .pipe(my_pipeline)
    .pipe(AddFiveStage())
)

# returns 36
# 5 * 3 = 15 * 2 + 1 = 31 + 5 = 36
await pipeline.process(5)

36

In [4]:
class MyPipeline(Pipeline[int]):
    stages = [
        TimesThreeStage(),
        my_pipeline,
        AddFiveStage(),
    ]

await MyPipeline().process(5)

36

In [5]:
###
## Stages with Custom Arguments
##
## On this example, the stages have the current index
## of the processing queue
###
from abc import abstractmethod
from typing import Awaitable, Callable, Concatenate

from thecodecrate_pipeline import (
    PipelineInterface,
    ProcessorInterface,
    StageInterface,
)
from thecodecrate_pipeline.core.pipeline.payload import TPayload


class IndexedStageInterface(StageInterface[TPayload]):
    @abstractmethod
    async def __call__(
        self,
        payload: TPayload,
        tag: int,
    ) -> TPayload:
        pass


IndexedPipelineCallable = (
    IndexedStageInterface[TPayload]
    | Callable[Concatenate[TPayload, ...], Awaitable[TPayload]]
    | Callable[Concatenate[TPayload, ...], TPayload]
)


class IndexedProcessor(ProcessorInterface[TPayload]):
    async def process(
        self,
        stages: list[IndexedPipelineCallable[TPayload]],
        payload: TPayload,
    ) -> TPayload:
        index = 0

        for stage in stages:
            payload = await self._call_stage(
                stage=stage,
                payload=payload,
                index=index,
            )

            index += 1

        return payload


class IndexedPipeline(PipelineInterface[TPayload]):
    processor_class = IndexedProcessor


###
## Usage
###
class MyIndexedStage(IndexedStageInterface[str]):
    # type-hinting: change `index` type to see an error
    async def __call__(self, payload: str, index: int) -> str:
        return f"{payload}: {index}"


indexed_pipeline = (
    (IndexedPipeline[str]()).pipe(MyIndexedStage()).pipe(MyIndexedStage())
)

assert await indexed_pipeline.process("test") == "test: 0: 1"

# returns "hello: 0: 1"
await indexed_pipeline.process("hello")

'hello: 0: 1'